In [1]:
import os 
import sys 
import json 
import logging 
from pathlib import Path 
from functools import cache
from itertools import product

# Required when developing in a jupyter-notebook environment 
cur_path = os.path.abspath("../..")
if cur_path not in sys.path: 
    sys.path.append(cur_path)

import numpy as np 
import pandas as pd 
import altair as alt 
from altair import datum
from palettable.tableau import GreenOrange_6
from dotenv import load_dotenv
from subgrounds.subgrounds import Subgrounds, Subgraph
from subgrounds.subgraph import SyntheticField
from subgrounds.pagination import ShallowStrategy

# Required when developing in a jupyter-notebook environment 
load_dotenv('../../../../.env')

print(os.environ['SUBGRAPH_URL'])
# logging.basicConfig(level=logging.DEBUG)

from utils_notebook.utils import (
    ddf, remove_prefix, load_subgraph, remove_keys, camel_to_snake_cols
)
from utils_notebook.vega import (
    output_chart, 
    apply_css, 
    wide_to_longwide, 
    chart
)
from utils_notebook.testing import validate_season_series
from utils_notebook.constants import ADDR_BEANSTALK
from utils_notebook.queries import QueryManager
from utils_notebook.css import css_tooltip_timeseries_multi_colored

https://api.thegraph.com/subgraphs/name/cujowolf/beanstalk


In [2]:
sg, bs = load_subgraph()
q = QueryManager(sg, bs) 

In [3]:
bs.SiloDailySnapshot.timestamp = bs.SiloDailySnapshot.createdAt
q = bs.Query.siloDailySnapshots(
    first=10000, orderBy="season", orderDirection="asc", where={'silo': ADDR_BEANSTALK}
)
df = sg.query_df([
    q.timestamp, q.seeds, q.stalk, q.depositedBDV
])
df = remove_prefix(df, "siloDailySnapshots_")
df.timestamp = pd.to_datetime(df.timestamp, unit='s')
df = df.sort_values('timestamp').reset_index(drop=True)
df = df.loc[(df.seeds > 0) & (df.stalk > 0)] 
df.seeds /= 1e6
df.stalk /= 1e10
df.depositedBDV /= 1e6 
df['seedsPerStalk'] = df.seeds / df.stalk 
df['stalkPerBdv'] = df.stalk / df.depositedBDV
df['seedsPerBdv'] = df.seeds / df.depositedBDV
df = camel_to_snake_cols(df)
df.head()

,timestamp,total_seeds,total_stalk,total_deposited_bdv,seeds_per_stalk,stalk_per_bdv,seeds_per_bdv
254,2022-08-04,1.173153e+08,4.068514e+07,3.231789e+07,2.883492,1.258905,3.630042
255,2022-08-05,1.193253e+08,4.101390e+07,3.264577e+07,2.909387,1.256331,3.655154
256,2022-08-06,1.221543e+08,4.169738e+07,3.337104e+07,2.929544,1.249508,3.660488
257,2022-08-07,1.525817e+08,5.000249e+07,4.213115e+07,3.051482,1.186829,3.621588
258,2022-08-08,1.568419e+08,5.189329e+07,4.340395e+07,3.022393,1.195589,3.613540


In [6]:
m1 = ['seeds', 'stalk', 'deposited_bdv']
m2 = ['seeds_per_stalk', 'stalk_per_bdv', 'seeds_per_bdv']
tooltip_metrics = m1 + m2 
data = wide_to_longwide(df, 'timestamp', ['timestamp'], tooltip_metrics)
tooltip_formats = {'seeds_per_stalk': '.2f', 'stalk_per_bdv': '.2f', 'seeds_per_bdv': '.2f'}
colors = {
    m: GreenOrange_6.hex_colors[i] for i, m in enumerate(tooltip_metrics)
}

chart_seed_stalk, selection_nearest = chart(
    data, 
    'timestamp', 
    lmetrics=m1,
    xaxis_kwargs=dict(labelSeparation=10), 
    yaxis_left_kwargs=dict(title=None, format=".2s"), 
    tooltip_metrics=tooltip_metrics, 
    tooltip_formats=tooltip_formats, 
    return_selection=True, 
    colors=colors, 
    width=400
)
chart_seeds_per_stalk = chart(
    data, 
    'timestamp', 
    lmetrics=m2, 
    yaxis_left_kwargs=dict(title=None), 
    tooltip_metrics=tooltip_metrics, 
    tooltip_formats=tooltip_formats, 
    selection_nearest=selection_nearest, 
    create_selection=False, 
    colors=colors,
    legend_kwargs=dict(title=None, orient="top"), 
    width=400
)
c = chart_seed_stalk | chart_seeds_per_stalk

css_lines = css_tooltip_timeseries_multi_colored(tooltip_metrics, colors)
css = '\n'.join(css_lines)
# apply_css(css)
apply_css("")
c

/Users/ALEX/opt/anaconda3/envs/beanstalk/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.HConcatChart(...)

In [5]:
output_chart(c, css=css)

<IPython.core.display.JSON object>